<a href="https://colab.research.google.com/github/26aharikrishnan/Next-Word-Telescope-Part-2/blob/main/Next_Word_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load pre-trained GPT-2 distilled model and tokenizer
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [2]:

# Parameters for poem generation
lines_target = 4          # Number of lines in the poem
min_words = 5             # Minimum words per line
max_words = 7             # Maximum words per line
top_k = 10                # Top-k sampling parameter for diversity
temperature = 1.0         # Sampling temperature for creativity

# Starting prompt for each line
starting_prompt = "The moon was shining bright at night, and the wind whispered softly"

# List to store each generated poem line
poem_lines = []

# Generate poem line-by-line
for _ in range(lines_target):
    current_text = starting_prompt  # Reset context for each new line
    current_line = ""               # Accumulate generated text for the current line
    word_count = 0                  # Track number of words generated for current line

    # Generate tokens until minimum word count reached (or max_words cap)
    while word_count < min_words:
        # Encode current context text as input IDs
        input_ids = tokenizer.encode(current_text, return_tensors="pt")
        attention_mask = torch.ones_like(input_ids)  # Attention mask (all tokens attended)

        # Generate multiple new tokens (3-5) at once for smoother output
        # Sampling with top-k and temperature adds controlled randomness and creativity
        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                attention_mask=attention_mask,
                do_sample=True,
                max_new_tokens=5,      # Generate up to 5 tokens per iteration
                top_k=top_k,
                temperature=temperature,
                pad_token_id=tokenizer.eos_token_id  # Ensure proper padding/eos handling
            )

        # Decode only the newly generated tokens (exclude prompt tokens)
        new_text = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

        # Ensure spacing between tokens for readability
        if not new_text.startswith(" "):
            new_text = " " + new_text

        # Append generated text to both full context and current line buffer
        current_text += new_text
        current_line += new_text

        # Update word count for the current line (splitting on whitespace)
        word_count = len(current_line.strip().split())

        # Stop early if maximum word count is reached
        if word_count >= max_words:
            break

    # Strip trailing spaces and add completed line to poem list
    poem_lines.append(current_line.strip())

# Output the generated poem with one line per poem line
print("\n".join(poem_lines))


into the air.
 

I was about
and gave a faint, sweet voice.
and softly to the other
in the background.
 The moon turned red and


In [19]:
pip install torch